# O nelinearnoj regresiji

## Regresija višeg stepena

U prethodnim vježbama smo uvidjeli značajnu razliku u kvalitetama linearne i kvadratne regresije, te postavili prirodno pitanje: Hoćemo li dobiti još bolje rezultate (tj. još manji MSE u našem slučaju) ukoliko dodatno povećamo stepen polinoma kojim računamo regresiju. Pogledajmo:

*Neophodne implementacije iz prethodnih vjezbi:*

In [1]:
from typing import List, Tuple

import plotly.offline as ply
import plotly.graph_objs as go
import numpy as np


# Instantiate dataset
car_mileage_vs_value_data_set: List[List[int]] = [
    (10000, 31000), (400000, 19000), (5000, 32000), (0, 40000),
    (1000, 33000), (100000, 26000), (50000, 29000), (50, 35000),
    (20000, 30000), (200000, 20000)]
car_mileage_vs_value_data_set: np.ndarray = np.array(car_mileage_vs_value_data_set)
car_mileages: np.ndarray = car_mileage_vs_value_data_set.T[0]
car_values: np.ndarray = car_mileage_vs_value_data_set.T[1]

# Instantiate plot.ly objects
ply.init_notebook_mode(connected=True)

plot_data = go.Scatter(x=car_mileages, 
                       y=car_values, 
                       mode='markers',
                       marker=dict(color='black'),
                       name='Training point')

layout = go.Layout(xaxis=dict(ticks='', showticklabels=True,
                              zeroline=False),
                   yaxis=dict(ticks='', showticklabels=True,
                              zeroline=False),
                   showlegend=False, hovermode='closest')


def calculate_polynomial_mse(training_set: List[Tuple[float, float]], *coefficients: float) -> float:
    polynomial = np.poly1d(coefficients)
    
    return sum([(polynomial(x) - y)**2 for x, y in training_set]) / len(training_set)


def get_regression_curve(input_data: np.ndarray, polynomial: np.poly1d, color: str = 'blue') -> go.Scatter:
    x_new = np.linspace(min(input_data), max(input_data), 50)
    y_new = polynomial(x_new)

    regression_curve = go.Scatter(
        x=x_new,
        y=y_new,
        mode='lines',
        line=dict(color=color, width=3),
        name='Regression curve')
    
    return regression_curve

Isprobajmo kubni polinom: $$p_3: y = m_3\cdot x^3 + m_2\cdot x^2 + m_1\cdot x + m_0$$ kao regresionu krivu.

In [2]:
m_3, m_2, m_1, m_0 = np.polyfit(
    x=car_mileages,
    y=car_values,
    deg=3)

print(f'm_3={m_3}, m_2={m_2}, m_1={m_1}, m_0={m_0}')
print(f'Cubic regression MSE: {calculate_polynomial_mse(car_mileage_vs_value_data_set, m_3, m_2, m_1, m_0)}')

figure = go.Figure(data=[plot_data, get_regression_curve(car_mileages, np.poly1d([m_3, m_2, m_1, m_0]))],
                   layout=layout)

ply.iplot(figure)

m_3=-4.4528879075042366e-13, m_2=4.226304878227773e-07, m_1=-0.13664585240154206, m_0=34494.49954539051
Cubic regression MSE: 4646173.626382839


Za nijansu bolji rezultat. *Prisjetimo se kvadratnog MSE-a sa prethodnih vježbi: $4816444$.* Šta ako probamo neki veći stepen, recimo 7.

In [3]:
coefficients = np.polyfit(
    x=car_mileages,
    y=car_values,
    deg=7)

print(f'Cubic regression MSE: {calculate_polynomial_mse(car_mileage_vs_value_data_set, *coefficients)}')

Cubic regression MSE: 1705216.6022341922


MSE značajno bolji! Hajde čisto iz kurioznosti da probamo još i stepen 10:

In [10]:
coefficients = np.polyfit(
    x=car_mileages,
    y=car_values,
    deg=10)

print(f'Cubic regression MSE: {calculate_polynomial_mse(car_mileage_vs_value_data_set, *coefficients)}')

Cubic regression MSE: 38.73987531533089


/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:4: RankWarning:

Polyfit may be poorly conditioned



MSE je skoro jednak nuli?! Je li to moguće? Pronašli smo model predikcije najbolje moguće kvalitete! Drugim riječima, sada za auto bilo koje kilometraže možemo izračunati skoro tačnu cijenu! Pogledajmo:
*(Opaska: Primijetite upozorenje `Polyfit may be poorly conditioned` prilikom izračuna koeficijenata. Osvrnuti ćemo se na njega kasnije.)*

In [8]:
polynomial = np.poly1d(coefficients)
test_car_mileages: np.ndarray = np.array([200000, 250000, 300000, 350000, 400000])
predicted_car_values: np.ndarray = polynomial(test_car_mileages)

for test_car_mileage, predicted_car_value in zip(test_car_mileages, predicted_car_values):
    print(f'Car mileage: {test_car_mileage} -> Car value: {round(predicted_car_value)}$')

Car mileage: 200000 -> Car value: 20000.0$
Car mileage: 250000 -> Car value: 19122506668816.0$
Car mileage: 300000 -> Car value: 124405892289612.0$
Car mileage: 350000 -> Car value: 278196507634565.0$
Car mileage: 400000 -> Car value: 143554933752392.0$


Hmm. Rezultati su, u najmanju ruku, čudni. Hajde da iscrtamo našu regresionu krivu:

In [11]:
figure = go.Figure(data=[plot_data, get_regression_curve(car_mileages, np.poly1d(coefficients), 'green')],
                   layout=layout)

ply.iplot(figure)

Sada i gore navedeno upozorenje `Polyfit may be poorly conditioned` ima smisla. Radi se upravo o upozorenju o velikoj razlici vrijednosti polinoma na različitim koordinatama. Npr. za tačku 50000 i 350000.

Primijetimo da se regresiona kriva “prilagodila" **islkjučivo** podacima u našem trening setu i na taj način reducirala grešku (u našem slučaju MSE) na 38 *(MSE bi, u teoriji, za stepen 10 trebao biti 0. Razmislite zašto. Zbog načina na koji `polyfit` računa koeficijente, te zbog greške zaokruživanja decimalnih vrijednosti, MSE poprima vrijednost nešto veću od nule.)*. Međutim, prilagođavajuci se našim podacima, neke vrijednosti (kao što je recimo tačka (350000, 1580127433543813)) su daleko promašene. Ovaj koncept, preprilagođavanja modela predikcije trening skupu, se naziva **overfitting-om** i to je jedan od najpoznatijih problema svih modela mašinskog učenja.